### Test sleep scoring
- Tried some unsupervised neural network from sklearn, it did not work that well

In [2]:
import numpy as np
from neuropy.analyses.brainstates import detect_brainstates_epochs
from neuropy.plotting import plot_probe
from neuropy.core import Epoch
from neuropy.utils.signal_process import FourierSg
from scipy.ndimage import gaussian_filter1d
from neuropy.analyses.brainstates import correlation_emg
from hfuncs import whiten_signal
from scipy import stats
from neuropy.analyses.brainstates import hmmfit1d
import matplotlib.pyplot as plt
from neuropy.plotting import plot_hypnogram
import subjects

smooth = lambda arr: gaussian_filter1d(arr, sigma=2 / 0.5)
# sess = subjects.SimData().ratSim1[0]
# sess = subjects.nsd.ratUday2[0]
# sess = subjects.nsd.ratSday2[0]
sess = subjects.sd.ratRday2[0]

### Spectrogram calculation

In [ ]:
from neuropy.utils.signal_process import FourierSg, hilbertfast, filter_sig

lfp = sess.eegfile.get_signal([sess.best_channels.slow_wave])
freqs = np.geomspace(1, 100, 100)
# freqs = np.(1, 100, 100)
spect = FourierSg(lfp, freqs=freqs, window=2, overlap=1, norm_sig=True)
log_spect = np.log10(spect.traces)
zsc_spect = stats.zscore(log_spect, axis=1)
time = spect.time

spect_noisy = stats.zscore(np.abs(zsc_spect.sum(axis=0))) >= 5

### Noisy indices

In [ ]:
noisy_bool = np.zeros_like(time).astype("bool")
if sess.artifact is not None:
    noisy_arr = sess.artifact.as_array()
    for noisy_ind in range(noisy_arr.shape[0]):
        st = noisy_arr[noisy_ind, 0]
        en = noisy_arr[noisy_ind, 1]
        noisy_bool[np.where((spect.time >= st - 1) & (spect.time <= en + 1))[0]] = True

noisy_bool = np.logical_or(spect_noisy, noisy_bool)

### Features for clustering

In [ ]:
from sklearn.decomposition import PCA
from numpy.polynomial.polynomial import polyfit

theta = spect.get_band_power(5, 10)
theta[~noisy_bool] = smooth(theta[~noisy_bool])
delta = spect.get_band_power(1, 4)
delta[~noisy_bool] = smooth(delta[~noisy_bool])

broadband_sw = np.zeros_like(time)
pca = PCA(n_components=2).fit(zsc_spect[:, ~noisy_bool].T)
# broadband_sw[~noisy_bool] = smooth(pca.transform(zsc_spect[:, ~noisy_bool].T).squeeze())
# broadband_sw[~noisy_bool] = smooth(pca.transform(zsc_spect[:, ~noisy_bool].T).squeeze())
pca_transform = pca.transform(zsc_spect[:, ~noisy_bool].T)

bp_2to16 = spect.get_band_power(2.16)
bp_2to16[~noisy_bool] = smooth(bp_2to16[~noisy_bool])
theta_ratio = theta / bp_2to16
theta_delta_ratio = theta / delta

emg = gaussian_filter1d(sess.emg.traces[0], sigma=10 * sess.emg.sampling_rate)
emg = np.log10(np.interp(spect.time, sess.emg.time, emg))
emg[np.isnan(emg)] = np.nanmax(emg)

theta_delta_emg_ratio = theta / (delta * 10**emg)

slope = polyfit(freqs, log_spect, deg=1)[1]

# Gervasoni et al. 2004

# bp1 = spect.get_band_ratio([0.5, 20], [0.5, 55])
# bp2 = spect.get_band_ratio([0.5, 4.5], [0.5, 9])

In [ ]:
_, ax = plt.subplots()
# ax.plot(spect.get_band_power(1, 4))
ax.plot(slope)

In [ ]:
_, axs = plt.subplots(3, 2, width_ratios=[4, 1])


def get_hist(arr):
    bins = np.linspace(arr.min(), arr.max(), 100)
    return np.histogram(arr, bins)


hist_delta, bins_delta = get_hist(delta)
hist_thratio, bins_thratio = get_hist(theta_ratio)
hist_emg, bins_emg = get_hist(emg)

# ax.plot(lfp_spect.time, theta_spect)
# ax.plot(lfp_spect.time, theta_hil)
axs[0, 0].plot(spect.time, delta)
axs[1, 0].plot(spect.time, theta_ratio)
axs[2, 0].plot(spect.time, emg)

axs[0, 1].plot(bins_delta[:-1], hist_delta)
axs[1, 1].plot(bins_thratio[:-1], hist_thratio)
axs[2, 1].plot(bins_emg[:-1], hist_emg)


# ax.axvline(sess.paradigm["maze"].flatten()[0])
# ax.axvline(sess.paradigm["maze"].flatten()[1])

### Clustering

In [ ]:
from neuropy.analyses.brainstates import gaussian_classify

_, axs = plt.subplots(1, 3, sharex=True, sharey=True)

# thratio_delta_arr = np.vstack((theta_ratio, broadband_sw)).T
thratio_delta_arr = np.vstack((theta, delta)).T

emg_bool = np.zeros_like(emg).astype("bool")
emg_bool[~noisy_bool] = gaussian_classify(emg[~noisy_bool]).astype("bool")

nrem_rem_bool = np.zeros_like(emg).astype("bool")
nrem_rem_bool[~emg_bool & ~noisy_bool] = gaussian_classify(
    theta_delta_ratio[~noisy_bool & ~emg_bool], ax=axs[0]
).astype("bool")

aw_qw_bool = np.zeros_like(emg).astype("bool")
aw_qw_bool[emg_bool & ~noisy_bool] = gaussian_classify(
    theta_ratio[~noisy_bool & emg_bool]
).astype("bool")

In [ ]:
label2 = gaussian_classify(theta_delta_ratio[~emg_bool & ~noisy_bool])

In [ ]:
np.sum([True, True])

In [ ]:
_, ax = plt.subplots()
ax.plot(emg_bool)

In [ ]:
_, ax = plt.subplots(2, sharex=True)

ax[0].plot(nrem_rem_bool[~noisy_bool & ~emg_bool])
ax[0].plot(label2.astype("bool") + 1.2)
ax[1].plot(theta_delta_ratio[~noisy_bool & ~emg_bool])

In [ ]:
_, ax = plt.subplots()

ax.plot(
    delta[~noisy_bool & ~emg_bool],
    theta[~noisy_bool & ~emg_bool],
    ".",
    alpha=0.5,
    markersize=2,
)

In [ ]:
_, ax = plt.subplots()

ax.plot(theta_ratio, delta, ".", markersize=1, alpha=0.5)

In [ ]:
from neuropy.analyses.brainstates import hmmfit1d

gmm_labels = gaussian_classify(theta_delta_emg_ratio[~noisy_bool & ~emg_bool])
hmm_labels = hmmfit1d(theta_delta_emg_ratio[~noisy_bool & ~emg_bool])

In [ ]:
from hmmlearn.hmm import GaussianHMM

feat = theta_delta_ratio[~noisy_bool & ~emg_bool][:, None]
g_model = GaussianHMM(n_components=2, n_iter=1).fit(feat)

In [ ]:
g_model.transmat_

In [ ]:
_, ax = plt.subplots(3, 1, sharex=True)

ax[0].plot(g_model)
ax[1].plot(p_model)


clash = np.where(np.logical_xor(hmm_labels, gmm_labels))[0]
ax[2].plot(theta_delta_emg_ratio[~noisy_bool & ~emg_bool])
ax[2].plot(clash, theta_delta_emg_ratio[~noisy_bool & ~emg_bool][clash], ".")

In [ ]:
np.corrcoef(hmm_labels, gmm_labels)

In [ ]:
_,ax = plt.subplots()

ax.plot(,'.')

### State labelling

In [ ]:
states = np.array([""] * len(theta_ratio), dtype="U4")
states[~noisy_bool & emg_bool & aw_qw_bool] = "AW"
states[~noisy_bool & emg_bool & ~aw_qw_bool] = "QW"
states[~noisy_bool & ~emg_bool & nrem_rem_bool] = "REM"
states[~noisy_bool & ~emg_bool & ~nrem_rem_bool] = "NREM"

states_epochs = Epoch.from_string_array(states)

### Combined plot

In [ ]:
from neuropy.plotting import plot_epochs

_, axs = plt.subplots(4, 1, sharex=True)


axs[0].plot(time[~noisy_bool], stats.zscore(delta[~noisy_bool]))
axs[0].plot(time[~noisy_bool], stats.zscore(broadband_sw[~noisy_bool]))
axs[1].plot(time[~noisy_bool], theta_ratio[~noisy_bool])
axs[2].plot(time[~noisy_bool], emg[~noisy_bool])


# axs[4].fill_between(states)
# plot_epochs(
#     ax=axs[3],
#     epochs=states_epochs,
#     labels_order=["NREM", "REM", "QW", "AW"],
#     colors=subjects.colors_sleep,
# )
axs[3].plot(time[~noisy_bool], (theta / delta)[~noisy_bool])
# axs[4].plot(high_emg)
# axs[5].plot((1/band30[:l])/delta_sxx[:-1])
# axs[5].plot(stats.zscore(delta_full))